In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import numpy as np
# notice we are adding two new imports for visualizations; This was not here last class
from itertools import product
#from pandas_profiling import ProfileReport

%matplotlib inline

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'


sns.set()

import dtale

In [101]:
df = pd.read_csv('C:/Users/vasco/OneDrive/Ambiente de Trabalho/Mestrado - Data Science/Data Mining/donors.csv')

C:\Users\vasco\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
df.head(10)

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A
5,5,2014-01-01,CWR,0,AL,35603,,,NaN,0,...,12.750000,47784,0,L,1,F,X,X,X,C
6,6,2007-01-01,DRK,0,IN,46755,,,1980-01-01,0,...,7.642857,62117,1,L,1,E,X,X,X,D
7,7,2014-01-01,NWN,0,LA,70611,,,NaN,0,...,6.200000,109359,1,L,3,E,X,X,X,C
8,8,2008-01-01,LIS,1,IA,51033,,,NaN,0,...,18.090909,75768,1,L,1,F,X,X,X,D
9,9,2014-01-01,MSD,1,TN,37127-,,,1952-11-01,0,...,9.333333,49909,0,L,1,F,X,X,X,B


In [102]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [5]:
# Categorical variables
cat_vars = [
    "STATE", # category
    
    "NOEXCH", # binary
    
    "RECINHSE", # binary
    #Donation in house
    
    "RECP3", # binary 
    #Public-Private Partnership
    
    "RECPGVG", # binary 
    # planned gift is a contribution that is arranged in the present and allocated at a future date. 
    #Commonly donated through a will or trust, planned gifts are most often granted once the donor has passed away.
    
    "RECSWEEP", # binary 
    #by sweepstakes
    
    "MAJOR", # binary
    "MDMAUD", # category
    "MDMAUD_R", # category
    "MDMAUD_F", # category
    "MDMAUD_A", # category
    
    "DOMAIN", # category
    "HOMEOWNR", # binary
    "CHILD03", # category
    "CHILD07", # category
    "CHILD12", # category
]

# Continuous variables
cont_vars = [
    "AGE", # 
    "",
]

# Variables for plots
plot_vars = [
    "STATE", 
    "ZIP", # clean zip [TODO]
    "GEOCODE2", 
    "ODATEDW", # turn into year-month-01 [TODO]
]

# not used features
drop_vars = [
    "GEOCODE", # 80k nan
    "MAILCODE", 
    "OSOURCE", 
    "DATASRCE",
    "PVASTATE",
    "LIFESRC",
]

# features to process
fe_features = [
    "TCODE", # Shorten it [TODO]
    "DOB", # Create age groups [DONE]
    "ODATEDW" # Create loyalty groups
]

# feature engineering
cat_fe_features = [
    "ageGroup", # category
    "loyaltyGroup", # category
]

cont_fe_features = [
    
]

In [6]:
# dataset data types
df.dtypes

ODATEDW     object
OSOURCE     object
TCODE        int64
STATE       object
ZIP         object
             ...  
RFA_2A      object
MDMAUD_R    object
MDMAUD_F    object
MDMAUD_A    object
GEOCODE2    object
Length: 475, dtype: object

In [7]:
# count of missing values
df.isna().sum()

ODATEDW       0
OSOURCE       0
TCODE         0
STATE         0
ZIP           0
           ... 
RFA_2A        0
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
GEOCODE2    132
Length: 475, dtype: int64

In [8]:
# duplicated observations
df.duplicated().sum()

0

In [9]:
#df.drop(columns = {'MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN',
#                  'PUBHLTH', 'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP'}, inplace = True)

In [10]:
# descriptive statistics
df.describe(include="all").T  # try with all and without all

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ODATEDW,95412,54,2015-01-01,15358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSOURCE,95412,896,MBC,4539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCODE,95412,NaN,NaN,NaN,54.2231,953.844,0,0,1,2,72002
STATE,95412,57,CA,17343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP,95412,19938,85351,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
RFA_2A,95412,4,F,46964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_R,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_F,95412,4,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_A,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.TCODE = df.TCODE.astype(object)
df['DOB'] = pd.to_datetime(df['DOB'])
df['ODATEDW'] = pd.to_datetime(df['ODATEDW'])

In [12]:
# check data types again
df.dtypes

ODATEDW     datetime64[ns]
OSOURCE             object
TCODE               object
STATE               object
ZIP                 object
                 ...      
RFA_2A              object
MDMAUD_R            object
MDMAUD_F            object
MDMAUD_A            object
GEOCODE2            object
Length: 475, dtype: object

In [13]:
# check descriptive statistics again
df.describe(include="all").T

<ipython-input-13-4f0fb0f0c755>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include="all").T
<ipython-input-13-4f0fb0f0c755>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include="all").T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
ODATEDW,95412,54,2015-01-01 00:00:00,15358,2003-06-01,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSOURCE,95412,896,MBC,4539,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCODE,95412,55,0,40917,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STATE,95412,57,CA,17343,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP,95412,19938,85351,61,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RFA_2A,95412,4,F,46964,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_R,95412,5,X,95118,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_F,95412,4,X,95118,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_A,95412,5,X,95118,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['MAILCODE'].value_counts()

     94013
B     1399
Name: MAILCODE, dtype: int64

In [17]:
df

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TIMELAG,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01 00:00:00,GRI,0,IL,61081,OK,,1957-12-01 00:00:00,0,,...,4,7.74194,0,L,4,E,X,X,X,C
148535,2014-01-01 00:00:00,BOA,1,CA,91326,OK,,1972-02-01 00:00:00,0,,...,18,15.6667,0,L,2,G,X,X,X,A
15078,2010-01-01 00:00:00,AMH,1,NC,27017,OK,,NaT,0,,...,12,7.48148,1,L,4,E,X,X,X,C
172556,2007-01-01 00:00:00,BRY,0,CA,95953,OK,,1948-01-01 00:00:00,0,,...,9,6.8125,1,L,4,E,X,X,X,C
7112,2006-01-01 00:00:00,,0,FL,33176,OK,,1940-01-01 00:00:00,0,X,...,14,6.86486,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184568,2016-01-01 00:00:00,ASE,1,AK,99504,OK,,NaT,0,,...,NaN,25,0,L,1,G,X,X,X,C
122706,2016-01-01 00:00:00,DCD,1,TX,77379,OK,,1970-01-01 00:00:00,0,,...,NaN,20,1,L,1,F,X,X,X,A
189641,2015-01-01 00:00:00,MBC,1,MI,48910,OK,,1958-01-01 00:00:00,0,,...,3,8.28571,1,L,3,E,X,X,X,B


In [16]:
df.loc[df['NOEXCH'] != '0']

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
901,2012-01-01,L03,0,MI,49740,,,1990-12-01,1,,...,10.062500,74361,0,L,2,F,X,X,X,D
955,2006-01-01,MCC,2,TN,37660,,,NaT,1,X,...,9.142857,13767,0,L,1,F,X,X,X,B
1133,2014-01-01,HOS,0,KS,67218,,,1970-01-01,1,,...,18.333333,105942,1,L,1,G,X,X,X,B
1340,2009-01-01,OMH,0,KS,66605,,,1958-01-01,1,,...,4.238095,105151,0,L,3,D,X,X,X,C
1378,2008-01-01,STR,2,FL,32440,,,1951-09-01,X,,...,10.461538,33734,0,L,1,F,X,X,X,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,ASE,1,AK,99504,,,NaT,0,,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,2006-01-01,PRV,0,CA,91320,,,1960-05-01,0,X,...,12.146341,4693,1,L,4,F,X,X,X,A


In [17]:
df.NOEXCH.replace({"X": True, "0": False, "1": False}, inplace=True)
df.RECINHSE.replace({"X": True, " ": False}, inplace=True)
df.RECP3.replace({"X": True, " ": False}, inplace=True)
df.RECPGVG.replace({"X": True, " ": False}, inplace=True)
df.RECSWEEP.replace({"X": True, " ": False}, inplace=True)


In [18]:
df['MDMAUD'].value_counts()

XXXX    93728
C1CM       64
L1CM       43
I1CM       36
D2CM       27
C2CM       24
D1CM       18
L2CM       15
C1LM        8
L1LM        7
I2CM        6
D5CM        5
D5MM        5
D2MM        4
C2LM        3
C5CM        3
I1LM        3
C2MM        3
D5TM        2
L2LM        1
C5TM        1
L2TM        1
C5MM        1
I5CM        1
C1MM        1
I5MM        1
L1MM        1
I2MM        1
Name: MDMAUD, dtype: int64

In [19]:
df['DOMAIN'].value_counts()

R2    13441
T2    12196
S1    11341
S2     8409
C2     8135
C1     6064
C3     5190
T1     4920
R3     4757
U1     4461
U2     3206
U3     2555
       2221
T3     2137
S3     1865
U4     1773
R1     1342
Name: DOMAIN, dtype: int64

In [20]:
df['HOMEOWNR'].value_counts()

H    51841
     21726
U    20446
Name: HOMEOWNR, dtype: int64

In [21]:
df['NUMCHLD'].value_counts()
df['NUMCHLD'].isna().sum()

81734

In [22]:
df['INCOME'].value_counts()
df['INCOME'].isna().sum()

20806

In [23]:
df = df.loc[(df['GENDER'] != 'C') & (df['GENDER'] != 'A')]
df['GENDER'].value_counts()

F    50514
M    38548
      2894
U     1691
J      362
Name: GENDER, dtype: int64

In [24]:
df['WEALTH1'].value_counts()

9.0    7506
8.0    6730
7.0    6132
6.0    5755
5.0    5217
4.0    4758
3.0    4180
2.0    4038
1.0    3412
0.0    2387
Name: WEALTH1, dtype: int64

In [25]:
df['WEALTH1'].isna().sum()

43894

In [26]:
df['HIT'].value_counts()

0     54680
1      8103
2      5562
3      3340
4      2791
      ...  
67        1
69        1
73        1
79        1
84        1
Name: HIT, Length: 75, dtype: int64

In [27]:
df['SOLP3'].value_counts()

      93839
00       73
12       69
01       18
02       10
Name: SOLP3, dtype: int64

In [28]:
df['SOLIH'].value_counts()

      87930
12     5593
00      280
01       93
02       72
03       18
04       16
06        7
Name: SOLIH, dtype: int64

In [29]:
df = df.loc[(df['SOLP3'] != '00') & (df['SOLIH'] != '00')]

In [30]:
df['MAJOR'].value_counts()

     93400
X      269
Name: MAJOR, dtype: int64

In [31]:
df.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [32]:
df['WEALTH2'].value_counts()

9.0    6398
8.0    5866
7.0    5567
6.0    5406
5.0    5230
4.0    4960
3.0    4919
2.0    4860
1.0    4094
0.0    3200
Name: WEALTH2, dtype: int64

In [33]:
df['WEALTH2'].isna().sum()

43169

In [34]:
df['GEOCODE2'].value_counts()

A    33830
B    27996
D    16291
C    15249
       182
Name: GEOCODE2, dtype: int64

In [103]:
metric = df.loc[:,(np.array((df.dtypes==int)|(df.dtypes==float)))]
non_metric= df.loc[:,(np.array((df.dtypes==object)))]

In [85]:
metric.columns

Index(['NUMCHLD', 'INCOME', 'WEALTH1', 'MBCRAFT', 'MBGARDEN', 'MBBOOKS',
       'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN',
       'PUBHLTH', 'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'WEALTH2',
       'MSA', 'ADI', 'DMA', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6',
       'RAMNT_7', 'RAMNT_8', 'RAMNT_9', 'RAMNT_10', 'RAMNT_11', 'RAMNT_12',
       'RAMNT_13', 'RAMNT_14', 'RAMNT_15', 'RAMNT_16', 'RAMNT_17', 'RAMNT_18',
       'RAMNT_19', 'RAMNT_20', 'RAMNT_21', 'RAMNT_22', 'RAMNT_23', 'RAMNT_24',
       'RAMNTALL', 'MINRAMNT', 'MAXRAMNT', 'LASTGIFT', 'TIMELAG', 'AVGGIFT'],
      dtype='object')

In [37]:
#non_metric.drop(columns = )

## Correlation

In [ ]:
fig = plt.figure(figsize=(15, 8))

# Obtain correlation matrix. Round the values to 2 decimal cases. Use the DataFrame corr() and round() method.
corr = np.round(df[metric].corr(method="pearson"), decimals=2)

# Build annotation matrix (values above |0.5| will appear annotated in the plot)
mask_annot = np.absolute(corr.values) >= 0.6
annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

# Plot heatmap of the correlation matrix
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

# Layout
fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

In [ ]:
#profile = ProfileReport(
#    df, 
 #   title='Tugas Customer Data',
 #   correlations={
  #      "pearson": {"calculate": True},
   #     "spearman": {"calculate": False},
    #    "kendall": {"calculate": False},
     #   "phi_k": {"calculate": False},
      #  "cramers": {"calculate": False},
   # },
#)

In [ ]:
#profile.to_notebook_iframe()

In [104]:
df.set_index('CONTROLN', inplace = True)

In [87]:
df

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TIMELAG,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,4.0,7.741935,0,L,4,E,X,X,X,C
148535,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,18.0,15.666667,0,L,2,G,X,X,X,A
15078,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,12.0,7.481481,1,L,4,E,X,X,X,C
172556,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,9.0,6.812500,1,L,4,E,X,X,X,C
7112,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,14.0,6.864865,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184568,2016-01-01,ASE,1,AK,99504,,,NaN,0,,...,NaN,25.000000,0,L,1,G,X,X,X,C
122706,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,,...,NaN,20.000000,1,L,1,F,X,X,X,A
189641,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,,...,3.0,8.285714,1,L,3,E,X,X,X,B


In [41]:
df['RAMNTALL'].value_counts()

20.00     4181
25.00     3646
15.00     3376
30.00     2536
40.00     1570
          ... 
42.25        1
136.07       1
77.25        1
365.01       1
75.55        1
Name: RAMNTALL, Length: 2051, dtype: int64

In [43]:
df['RAMNTALL'].isna().sum()

0

In [49]:
df['NGIFTALL'].value_counts()

1     9908
2     7616
3     7077
4     6914
5     6203
      ... 
71       1
72       1
76       1
73       1
87       1
Name: NGIFTALL, Length: 88, dtype: int64

In [50]:
df['CARDGIFT'].value_counts()

1     17294
2     13142
3     10276
4      8272
5      6633
6      5913
7      5007
0      4916
8      4224
9      3431
10     2951
11     2441
12     2043
13     1616
14     1327
15      963
16      734
17      604
18      451
19      377
20      280
21      225
22      161
23      111
24      104
25       75
26       49
27       20
28        9
29        9
30        3
33        2
36        2
31        2
32        1
41        1
Name: CARDGIFT, dtype: int64

In [51]:
#!pip install dtale

You should consider upgrading via the 'C:\Users\vasco\anaconda3\python.exe -m pip install --upgrade pip' command.



  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669701 sha256=a809e82b78d8f3e1542d3ae341f8a2e3422ce12cc8d63f18ab64164ac9cd6ccc
  Stored in directory: c:\users\vasco\appdata\local\pip\cache\wheels\00\35\e5\57e90f10c529601c6627400513cb65dd5adb09752411f5a050
  Created wheel for ppscore: filename=ppscore-1.1.1-py2.py3-none-any.whl size=12988 sha256=dd56f3629e0d7b9725038c1aea6d4c38413a1f72719f7ed95cd0a679e6a2f487
  Stored in directory: c:\users\vasco\appdata\local\pip\cache\wheels\e1\45\ce\6bb5da9f22f466ba4e0ef6e3a3d2bbcde992f96c550165fc60
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-py3-none-any.whl size=62598 sha256=1dee27a1f075b9e669fdca379ba3d1c12ba80d92997bb806b35c67320d1916e2
  Stored in directory: c:\users\vasco\appdata\local\pip\cache\wheels\fe\33\33\172dbfe1efc739352828854a113df599a428b58f68ed0c4f75
  Created wheel for dash: filename=dash-1.18.1-py3-none-any.whl size=83679 sha256=044b4d365690932f5123bfde12972529ba71a15c1cfb8da65

In [105]:
#1399 records have a bad addres, we should drop it
df['MAILCODE'].replace(' ', 'OK', inplace = True)

In [106]:
df['OSOURCE'].replace(' ', np.nan, inplace = True)
df['PVASTATE'].replace(' ', np.nan, inplace = True)
df['DOMAIN'].replace(' ', np.nan, inplace = True)
df['HOMEOWNR'].replace(' ', np.nan, inplace = True)
df['GENDER'].replace(' ', np.nan, inplace = True)
df['DATASRCE'].replace(' ', np.nan, inplace = True)
df['GEOCODE'].replace(' ', np.nan, inplace = True)
df['LIFESRC'].replace(' ', np.nan, inplace = True)
df['RFA_5'].replace(' ', np.nan, inplace = True)
df['RFA_7'].replace(' ', np.nan, inplace = True)
df['RFA_3'].replace(' ', np.nan, inplace = True)
df['RFA_4'].replace(' ', np.nan, inplace = True)
df['RFA_2'].replace(' ', np.nan, inplace = True)
df['RFA_6'].replace(' ', np.nan, inplace = True)
df['RFA_8'].replace(' ', np.nan, inplace = True)
df['RFA_9'].replace(' ', np.nan, inplace = True)
df['RFA_10'].replace(' ', np.nan, inplace = True)
df['RFA_11'].replace(' ', np.nan, inplace = True)
df['RFA_12'].replace(' ', np.nan, inplace = True)
df['RFA_13'].replace(' ', np.nan, inplace = True)
df['RFA_14'].replace(' ', np.nan, inplace = True)
df['RFA_15'].replace(' ', np.nan, inplace = True)
df['RFA_16'].replace(' ', np.nan, inplace = True)
df['RFA_17'].replace(' ', np.nan, inplace = True)
df['RFA_18'].replace(' ', np.nan, inplace = True)
df['RFA_19'].replace(' ', np.nan, inplace = True)
df['RFA_20'].replace(' ', np.nan, inplace = True)
df['RFA_21'].replace(' ', np.nan, inplace = True)
df['RFA_22'].replace(' ', np.nan, inplace = True)
df['RFA_23'].replace(' ', np.nan, inplace = True)
df['RFA_24'].replace(' ', np.nan, inplace = True)
df['GEOCODE2'].replace(' ', np.nan, inplace = True)
df['RECINHSE'].replace(' ', 'O', inplace = True)
df['RECP3'].replace(' ', 'O', inplace = True)
df['RECPGVG'].replace(' ', 'O', inplace = True)
df['RECSWEEP'].replace(' ', 'O', inplace = True)
df['SOLP3'].replace(' ', 'x', inplace = True)
df['SOLIH'].replace(' ', 'x', inplace = True)
df['MAJOR'].replace(' ', 'O', inplace = True)

In [75]:
dtale.show(df, ignore_duplicate = True)

Exception occurred while processing request: multi-line expressions are only valid in the context of data, use DataFrame.eval
Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\views.py", line 83, in _handle_exceptions
    return func(*args, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\views.py", line 1503, in test_filter
    run_query(
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\query.py", line 64, in run_query
    df = df.query(final_query, local_dict=context_vars or {})
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\frame.py", line 3341, in query
    res = self.eval(expr, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\frame.py", line 3471, in eval
    return _eval(expr, inplace=inplace, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\eval.py", line 316, in eval
    raise ValueError(
ValueError: multi-line expressions are only vali

2020-12-31 16:17:50,838 - ERROR    - Exception occurred while processing request: multi-line expressions are only valid in the context of data, use DataFrame.eval
Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\views.py", line 83, in _handle_exceptions
    return func(*args, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\views.py", line 1503, in test_filter
    run_query(
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\query.py", line 64, in run_query
    df = df.query(final_query, local_dict=context_vars or {})
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\frame.py", line 3341, in query
    res = self.eval(expr, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\frame.py", line 3471, in eval
    return _eval(expr, inplace=inplace, **kwargs)
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\eval.py", line 316, in eval
    raise ValueError(
ValueError:

Exception occurred while processing request: name 'row' is not defined
Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\scope.py", line 188, in resolve
    return self.resolvers[key]
  File "C:\Users\vasco\anaconda3\lib\collections\__init__.py", line 898, in __getitem__
    return self.__missing__(key)            # support subclasses that define __missing__
  File "C:\Users\vasco\anaconda3\lib\collections\__init__.py", line 890, in __missing__
    raise KeyError(key)
KeyError: 'row'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\scope.py", line 199, in resolve
    return self.temps[key]
KeyError: 'row'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\dtale\views.py", line 83, in _handle_ex

2020-12-31 16:18:30,572 - ERROR    - Exception occurred while processing request: name 'row' is not defined
Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\scope.py", line 188, in resolve
    return self.resolvers[key]
  File "C:\Users\vasco\anaconda3\lib\collections\__init__.py", line 898, in __getitem__
    return self.__missing__(key)            # support subclasses that define __missing__
  File "C:\Users\vasco\anaconda3\lib\collections\__init__.py", line 890, in __missing__
    raise KeyError(key)
KeyError: 'row'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\pandas\core\computation\scope.py", line 199, in resolve
    return self.temps[key]
KeyError: 'row'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\vasco\anaconda3\lib\site-packages\dt

In [107]:
percent_missing = df.isna().sum() * 100 / len(df)
number_missing = df.isna().sum()
missing_value_df = pd.DataFrame({'number_missing': number_missing,
                                 'percent_missing': percent_missing})

In [108]:
missing_value_df.loc[missing_value_df['percent_missing']>=40]

,number_missing,percent_missing
PVASTATE,93954,98.471890
NUMCHLD,83026,87.018404
WEALTH1,44732,46.882992
MBCRAFT,52854,55.395548
MBGARDEN,52854,55.395548
...,...,...
RAMNT_20,87524,91.732696
RAMNT_21,85899,90.029556
RAMNT_22,74539,78.123297
RAMNT_23,87553,91.763091


In [109]:
#pd.set_option('display.max_rows', df.shape[0]+1)

In [ ]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1

outliers_indexes = []
lower_lim = q1 - 1.5*iqr
upper_lim = q3 + 1.5*iqr

for m in metric:
    llim = lower_lim[m]
    ulim = upper_lim[m]

    for val in df[m]:
        if ((val < llim) | (val > ulim)):
            outliers_indexes.append(df[df[m] == val].index.values)
            
outliers_indexes